In [30]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import os, sys
import datetime as dt

In [32]:
main_data = pd.read_csv("C:\Boot Camp\Homework\project3\CSV export\Main_table.csv")
main_data.head()

,visitNumber,visitId,visitStartTime,date,fullVisitorId,channelGrouping,socialEngagementType,visits,hits,pageviews,...,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,city,networkDomain
0,1,1478008845,1478008845,20161101,9352881518669559847,Organic Search,Not Socially Engaged,1,1,1.0,...,Chrome,Android,True,mobile,Europe,Western Europe,Germany,Hamburg,Hamburg,(not set)
1,1,1478002133,1478002133,20161101,4906813510055416703,Organic Search,Not Socially Engaged,1,1,1.0,...,Chrome,Windows,False,desktop,Asia,Eastern Asia,Taiwan,Taipei City,(not set),hinet.net
2,1,1477995977,1477995977,20161101,4091646801425702074,Organic Search,Not Socially Engaged,1,1,1.0,...,Chrome,Windows,False,desktop,Europe,Southern Europe,Spain,not available in demo dataset,not available in demo dataset,comunitel.net
3,1,1478034201,1478034201,20161101,5401793363522001290,Direct,Not Socially Engaged,1,1,1.0,...,Opera Mini,Android,True,mobile,Africa,Western Africa,Niger,not available in demo dataset,not available in demo dataset,(not set)
4,1,1478055650,1478055650,20161101,4440833520759626888,Organic Search,Not Socially Engaged,1,1,1.0,...,Chrome,Linux,False,desktop,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,utah.edu


In [33]:
clean_main_data=main_data[["visitId", "fullVisitorId", "date", "channelGrouping", "hits", "timeOnSite", "browser", "operatingSystem", "deviceCategory", "continent", "country", "region", "city", "networkDomain"]].copy()
clean_main_data.head()

,visitId,fullVisitorId,date,channelGrouping,hits,timeOnSite,browser,operatingSystem,deviceCategory,continent,country,region,city,networkDomain
0,1478008845,9352881518669559847,20161101,Organic Search,1,NaN,Chrome,Android,mobile,Europe,Germany,Hamburg,Hamburg,(not set)
1,1478002133,4906813510055416703,20161101,Organic Search,1,NaN,Chrome,Windows,desktop,Asia,Taiwan,Taipei City,(not set),hinet.net
2,1477995977,4091646801425702074,20161101,Organic Search,1,NaN,Chrome,Windows,desktop,Europe,Spain,not available in demo dataset,not available in demo dataset,comunitel.net
3,1478034201,5401793363522001290,20161101,Direct,1,NaN,Opera Mini,Android,mobile,Africa,Niger,not available in demo dataset,not available in demo dataset,(not set)
4,1478055650,4440833520759626888,20161101,Organic Search,1,NaN,Chrome,Linux,desktop,Americas,United States,not available in demo dataset,not available in demo dataset,utah.edu


In [39]:
clean_main_data.to_csv("clean_main_data.csv", index=False)

In [41]:
clean_main_data.count()

visitId            113971
fullVisitorId      113971
date               113971
channelGrouping    113971
hits               113971
timeOnSite          56353
browser            113971
operatingSystem    113971
deviceCategory     113971
continent          113971
country            113971
region             113971
city               113971
networkDomain      113971
dtype: int64

In [94]:
grouped_data=main_data.groupby("visitId")
Nov_device_count=grouped_data["deviceCategory"].head().value_counts()
total_counts=main_data["visitId"].count()
Nov_percentage=[]
for a in Nov_device_count:
    result=round(a/total_counts*100,2)
    Nov_percentage.append(result)
#device_count.values
Nov_devices=pd.DataFrame({"Device Name":device_count.keys()})
Nov_devices.head()
Nov_percentage

list